In [ ]:
# %load_ext autoreload
# %autoreload 2


In [ ]:
import time

import ray
import ray.util.scheduling_strategies

import celltrip


In [ ]:
ray.shutdown()
ray.init(
    address='ray://127.0.0.1:10001',
    runtime_env={
        'env_vars': {
            'RAY_DEDUP_LOGS': '0',
            # Irregular node fixes
            # NOTE: Important, NCCL will timeout if network device is non-standard
            # 'CUDA_LAUNCH_BLOCKING': '1',  # Slow, only for compatibility with X windows
            # 'NCCL_SOCKET_IFNAME': 'tailscale',  # lo,en,wls,docker,tailscale
            # 'NCCL_IB_DISABLE': '1',
            # 'NCCL_CUMEM_ENABLE': '0',
            # 'NCCL_DEBUG': 'INFO',
        }
    }
)


In [ ]:
def env_init(parent_dir=False):
    # Create dataloader
    fnames = ['./data/MERFISH/expression.h5ad', './data/MERFISH/spatial.h5ad']
    if parent_dir: fnames = ['.' + f for f in fnames]
    partition_cols = None  # 'layer'
    adatas = celltrip.utility.processing.read_adatas(*fnames, on_disk=False)
    celltrip.utility.processing.test_adatas(*adatas, partition_cols=partition_cols)
    dataloader = celltrip.utility.processing.PreprocessFromAnnData(
        *adatas, partition_cols=partition_cols,  num_nodes=200,
        pca_dim=128, seed=42)
    # modalities, adata_obs, adata_vars = dataloader.sample()
    # Return env
    return celltrip.environment.EnvironmentBase(dataloader, dim=3, penalty_bound=1, reward_origin=1)

# Default 25Gb Forward, 14Gb Update, at max capacity
policy_init = lambda env: celltrip.policy.PPO(
    2*env.dim, env.dataloader.modal_dims, env.dim,
    forward_batch_size=int(5e5)) # update_iterations=2, minibatch_size=3e3,

memory_init = lambda policy: celltrip.memory.AdvancedMemoryBuffer(
    sum(policy.modal_dims), split_args=policy.split_args)

initializers = (policy_init, env_init, memory_init)


In [ ]:
start = time.perf_counter()
# workers = ray.get(celltrip.train.remote(2, 2, 4, rollout_kwargs={'dummy': True}, update_kwargs={'update_iterations': 5}))
# workers = ray.get(celltrip.train.remote(2, 2, 4, sync_across_nodes=False))  # sync_across_nodes=False
workers = ray.get(celltrip.train.train_celltrip.remote(2, 1, 4, initializers))
print(time.perf_counter() - start)


In [ ]:
# env = env_init(parent_dir=True).to('cuda')
# policy = policy_init(env).to('cuda')
# memory = memory_init(policy)
# celltrip.train.simulate_until_completion(policy, env, memory)
# memory.propagate_rewards()
# memory.normalize_rewards()
# # for _ in range(5):
# #     memory.append_memory(memory)
# len(memory)
# # memory.fast_sample(10_000, shuffle=False)
